In [ ]:
! git clone https://github.com/scrollmapper/bible_databases.git

Cloning into 'bible_databases'...
remote: Enumerating objects: 1585, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 1585 (delta 19), reused 40 (delta 11), pack-reused 1521
Receiving objects: 100% (1585/1585), 164.14 MiB | 20.58 MiB/s, done.
Resolving deltas: 100% (503/503), done.
Checking out files: 100% (800/800), done.


In [ ]:
cd /content/bible_databases/

/content/bible_databases


## Import Libraries

In [2]:
import re
import requests
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from glob import glob
import nltk

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pmcbe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pmcbe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pmcbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\pmcbe\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [4]:
# These stopwords are commonly used words in the English language
STOPWORDS = {re.sub(r'[^\w\s]', '', word) for word in stopwords.words('english')} #this line is one I found online 
# how to add other stopwords
STOPWORDS.add("microelectronical")

## Import TOKENS, VOCAB, and LIB

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/DS5001/Proj/

/content/drive/MyDrive/DS5001/Proj


In [5]:
data_in = './data_in/'

In [6]:
VOCAB_bible = pd.read_csv('{}/bible_vocab.csv'.format(data_in)).set_index('term_str')
VOCAB_bible.head()

,term_id,n,stop
term_str,,,
the,0,58763,1
and,1,51839,1
of,2,34239,1
to,3,13646,1
that,4,13498,1


In [19]:
bible_tokens = pd.read_csv('{}/bible_tokens.csv'.format(data_in)).set_index(['author','book','sent','token_num'])
bible_tokens['term_str'] = bible_tokens.token_str.replace(r'\W+', '', regex=True).str.lower()
bible_tokens.head()

pos  token_str   term_str
author book    sent token_num                           
God    Genesis 0    0           IN         In         in
                    1           DT        the        the
                    2           NN  beginning  beginning
                    3          NNP        God        god
                    4          VBD    created    created

In [28]:
speeches_tokens = pd.read_csv('{}/speeches_tokens.csv'.format(data_in)).set_index(['author','book','sent','token_num'])
print(speeches_tokens.shape)
speeches_tokens.head()

(4022103, 3)


pos token_str term_str
author           book                   sent token_num                        
William McKinley  Second Annual Message 0    0           TO        To       to
                                             1           DT       the      the
                                             2          NNP    Senate   senate
                                             3           CC       and      and
                                             4          NNP     House    house

In [29]:
speeches_tokens = speeches_tokens.reset_index()
speeches_tokens.author.unique()

array(['William McKinley', 'Grover Cleveland', 'Ulysses S. Grant',
       'George Washington', 'Jimmy Carter', 'Ronald Reagan',
       'Gerald Ford', 'Woodrow Wilson', 'Calvin Coolidge',
       'Warren G. Harding', 'Chester A. Arthur', 'Thomas Jefferson',
       'John Adams', 'Andrew Jackson', 'John Quincy Adams',
       'James Monroe', 'Lyndon B. Johnson', 'Dwight D. Eisenhower',
       'Harry S. Truman', 'Richard M. Nixon', 'George H. W. Bush',
       'Franklin D. Roosevelt', 'John F. Kennedy', 'George W. Bush',
       'Bill Clinton', 'James Madison', 'Barack Obama',
       'James A. Garfield', 'Rutherford B. Hayes', 'Franklin Pierce',
       'Abraham Lincoln', 'John Tyler', 'William Harrison',
       'Benjamin Harrison', 'Andrew Johnson', 'Herbert Hoover',
       'James K. Polk', 'James Buchanan', 'Joe Biden', 'Donald Trump',
       'William Taft', 'Theodore Roosevelt', 'Martin Van Buren',
       'Zachary Taylor', 'Millard Fillmore'], dtype=object)

In [31]:
speechTitles = list(speeches_tokens.book.unique())
noDebates = [title for title in speechTitles if 'debate' not in title.lower()]
speeches_tokens = speeches_tokens[speeches_tokens['book'].isin(noDebates)]

In [32]:
presidents = ['Jimmy Carter', 'Ronald Reagan',
       'Richard M. Nixon', 'George H. W. Bush',
       'Franklin D. Roosevelt', 'John F. Kennedy', 'George W. Bush',
       'Bill Clinton', 'Barack Obama', 'Donald Trump']
speeches_tokens = speeches_tokens[speeches_tokens['author'].isin(presidents)]
speeches_tokens = speeches_tokens.set_index(['author','book','sent','token_num'])
speeches_tokens.head()

pos  \
author        book                                           sent token_num        
Ronald Reagan  Remarks at the Republican National Convention 0    0           NN   
                                                             1    0          NNP   
                                                                  1          NNP   
                                                             2    0          NNP   
                                                                  1          NNP   

                                                                              token_str  \
author        book                                           sent token_num               
Ronald Reagan  Remarks at the Republican National Convention 0    0                  Mr   
                                                             1    0          President,   
                                                                  1                 Mrs   
                                                             2    0               Ford,   
                                                                  1                  Mr   

                                                                              term_str  
author        book                                           sent token_num             
Ronald Reagan  Remarks at the Republican National Convention 0    0                 mr  
                                                             1    0          president  
                                                                  1                mrs  
                                                             2    0               ford  
                                                                  1                 mr

In [33]:
print(speeches_tokens.shape,bible_tokens.shape)
bible_tokens.head()

(1266645, 3) (780437, 3)


pos  token_str   term_str
author book    sent token_num                           
God    Genesis 0    0           IN         In         in
                    1           DT        the        the
                    2           NN  beginning  beginning
                    3          NNP        God        god
                    4          VBD    created    created

In [34]:
TOKENS_all = pd.concat([speeches_tokens, bible_tokens],axis=0)
TOKENS_all.head()

pos  \
author        book                                           sent token_num        
Ronald Reagan  Remarks at the Republican National Convention 0    0           NN   
                                                             1    0          NNP   
                                                                  1          NNP   
                                                             2    0          NNP   
                                                                  1          NNP   

                                                                              token_str  \
author        book                                           sent token_num               
Ronald Reagan  Remarks at the Republican National Convention 0    0                  Mr   
                                                             1    0          President,   
                                                                  1                 Mrs   
                                                             2    0               Ford,   
                                                                  1                  Mr   

                                                                              term_str  
author        book                                           sent token_num             
Ronald Reagan  Remarks at the Republican National Convention 0    0                 mr  
                                                             1    0          president  
                                                                  1                mrs  
                                                             2    0               ford  
                                                                  1                 mr